In [4]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [3]:
# Import data MNIST
mnist_data = tfds.load("mnist")
mnist_train, mnist_test = mnist_data["train"], mnist_data["test"]

Dataset mnist downloaded and prepared to C:\Users\antonio.martinez2\tensorflow_datasets\mnist\1.0.0. Subsequent calls will reuse this data.


#### Datasetbuilder
* Where to download the data from and how to extract it and write it to a standard format (DatasetBuilder.download_and_prepare)
* How to load it from disk (DatasetBuilder.as_dataset).
* And all the information about the dataset, like the names, types, and shapes of all the features, the number of records in each split, the source URLs, citation for the dataset or associated paper, etc. (DatasetBuilder.info).

In [6]:
# Fetch the dataset directly
mnist = tfds.image.MNIST()

# or by string name 
mnist = tfds.builder("mnist")

# Describe the dataset with dataserInfo
assert mnist.info.features['image'].shape == (28, 28, 1)
assert mnist.info.features['label'].num_classes == 10
assert mnist.info.splits['train'].num_examples == 60000

In [7]:
# Download the data, prepare it and write to disk
mnist.download_and_prepare()

# Load data from disk as tf.data.Datasets
datasets = mnist.as_dataset()
train_dataset, test_dataset = datasets['train'], datasets['test']
assert isinstance(train_dataset, tf.data.Dataset)

In [13]:
# Convert the Dataset to NumPy arrays
for example in tfds.as_numpy(train_dataset):
    image, label = example['image'], example['label']
    #assert isinstance(image, np.array)

As a convenience, you can do all the above with tfds.load, which fetches the DatasetBuilder by name, calls download_and_prepare(), and calls as_dataset().

In [26]:
datasets = tfds.load("mnist")
train_dataset, test_dataset = datasets['train'], datasets['test']
assert isinstance(train_dataset, tf.data.Dataset)

#### Dataset configuration
Datasets with different variants are configured with named BuilderConfigs. 

In [27]:
# Build-in configs
configs = tfds.text.IMDBReviews.builder_configs
assert "bytes" in configs

In [30]:
# Address a built-in config with tfds.builder
imdb = tfds.builder("imdb_reviews/bytes")
# or when constructing the builder directly
imdb = tfds.text.IMDBReviews(config="bytes")
# or use your own custom configuration
my_encoder = tfds.features.text.ByteTextEncoder(additional_tokens=['hello'])
my_config = tfds.text.IMDBReviewsConfig(
    name="my_config",
    version="1.0.0",
    text_encoder_config=tfds.features.text.TextEncoderConfig(encoder=my_encoder),
)
imdb = tfds.text.IMDBReviews(config=my_config)

W0719 10:26:15.948255 21304 dataset_builder.py:540] Using custom data configuration my_config


#### Text datasets and vocabulary 
The encoders, along with their vocabulary sizes, can be accessed through DatasetInfo:

In [33]:
imdb = tfds.builder("imdb_reviews/subwords8k")

# Get the TextEncoder from DatasetInfo
encoder = imdb.info.features["text"].encoder
assert isinstance(encoder, tfds.features.text.SubwordTextEncoder)

# Encoder, decode
ids = encoder.encode("Hello world")
assert encoder.decode(ids) == "Hello world"

# Get the vocabulary size
vocab_size = encoder.vocab_size